In [19]:
import os
import glob
import pandas as pd

# =====================================================
# Step 1: Load & Preprocess All CSVs
# =====================================================

base_dir = r"C:\Users\james\OneDrive\Documents\GitHub\solana-qrf-interval-forecasting\data"
csv_paths = {
    "tokens_map":         os.path.join(base_dir, "tokens.csv"),
    "holders":            os.path.join(base_dir, "holders.csv"),
    "new_token_accounts": os.path.join(base_dir, "new_token_accounts.csv"),
    "token_transfers":    os.path.join(base_dir, "token_transfers.csv"),
    "btc_eth_price":      os.path.join(base_dir, "btc_eth_price.csv"),
    "sol_price":          os.path.join(base_dir, "sol_price.csv"),
    "network_tx":         os.path.join(base_dir, "network_transaction_counts.csv"),
    "tvl":                os.path.join(base_dir, "tvl.csv"),
    "spl_instructions":   os.path.join(base_dir, "SPL Program Instructions.csv"),
}
token_files = glob.glob(os.path.join(base_dir, "tokens", "*.csv"))

# 1.1 tokens.csv → symbol_to_mint
df_tokens_map = pd.read_csv(csv_paths["tokens_map"])
if "token_symbol" not in df_tokens_map.columns:
    df_tokens_map.rename(columns={"symbol":"token_symbol"}, inplace=True)
if "token_mint" not in df_tokens_map.columns:
    df_tokens_map.rename(columns={"mint":"token_mint"}, inplace=True)
df_tokens_map = df_tokens_map.drop_duplicates(["token_symbol","token_mint"])
df_tokens_map["token_mint"] = df_tokens_map["token_mint"].str.lower()
symbol_to_mint = df_tokens_map.set_index("token_symbol")["token_mint"].to_dict()
mint_to_symbol = {m:s for s,m in symbol_to_mint.items()}

# 1.2 holders.csv
df_holders = pd.read_csv(csv_paths["holders"])
df_holders.rename(columns={c:"timestamp" for c in ["time","date","datetime","bucket_start"] if c in df_holders}, inplace=True)
df_holders["timestamp"] = pd.to_datetime(df_holders["timestamp"], utc=True).dt.tz_convert(None).dt.floor("12h")
df_holders["token_mint"] = df_holders["token_mint"].str.lower()
df_holders = df_holders[["timestamp","token_mint","holder_count"]]

# 1.3 new_token_accounts.csv
df_newacc = pd.read_csv(csv_paths["new_token_accounts"])
df_newacc.rename(columns={"bucket_start":"timestamp"}, inplace=True)
df_newacc.rename(columns={"token_mint_address":"token_mint"}, inplace=True)
df_newacc["timestamp"] = pd.to_datetime(df_newacc["timestamp"], utc=True).dt.tz_convert(None).dt.floor("12h")
df_newacc["token_mint"] = df_newacc["token_mint"].str.lower()
df_newacc = df_newacc[["timestamp","token_mint","new_token_accounts"]]

# 1.4 token_transfers.csv
df_transfers = pd.read_csv(csv_paths["token_transfers"])
df_transfers.rename(columns={"bucket_start":"timestamp"}, inplace=True)
df_transfers.rename(columns={"token_mint_address":"token_mint"}, inplace=True)
df_transfers["timestamp"] = pd.to_datetime(df_transfers["timestamp"], utc=True).dt.tz_convert(None).dt.floor("12h")
df_transfers["token_mint"] = df_transfers["token_mint"].str.lower()
for col in ["unique_senders","unique_receivers"]:
    if col in df_transfers: df_transfers.drop(columns=col,inplace=True)
df_transfers = df_transfers[["timestamp","token_mint","transfer_count"]]

# 1.5 global CSVs (floor to 12H)
def load_and_floor(path, rename_map=None):
    df = pd.read_csv(path)
    if rename_map:
        df.rename(columns=rename_map, inplace=True)
    df.rename(columns={c:"timestamp" for c in ["time","date","datetime","bucket_start"] if c in df}, inplace=True)
    df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True).dt.tz_convert(None).dt.floor("12h")
    return df

df_btceth = load_and_floor(csv_paths["btc_eth_price"])
df_sol    = load_and_floor(csv_paths["sol_price"])
# network_tx: detect bucket_start
df_net = pd.read_csv(csv_paths["network_tx"])
df_net.rename(columns={"bucket_start":"timestamp"}, inplace=True)
df_net["timestamp"] = pd.to_datetime(df_net["timestamp"], utc=True).dt.tz_convert(None).dt.floor("12h")
df_tvl = load_and_floor(csv_paths["tvl"])
df_spl = pd.read_csv(csv_paths["spl_instructions"])

# 1.6 per-token OHLCV
dfs_tokens = {}
for path in token_files:
    symbol = os.path.splitext(os.path.basename(path))[0]
    df = pd.read_csv(path)
    if "holder_count" in df: df.drop(columns="holder_count", inplace=True)
    df.rename(columns={c:"timestamp" for c in ["time","date","datetime"] if c in df}, inplace=True)
    df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True).dt.tz_convert(None).dt.floor("12h")
    df["token_symbol"] = symbol
    df["token_mint"] = symbol_to_mint[symbol]
    dfs_tokens[symbol] = df

# =====================================================
# Step 2: Build 12H Index & Align
# =====================================================

# Normalize & dedup global
globals_ = {"btc_eth_price":df_btceth,"sol_price":df_sol,"network_tx":df_net,"tvl":df_tvl}
for k,df in globals_.items():
    df.sort_values("timestamp",inplace=True)
    df.drop_duplicates("timestamp",keep="last",inplace=True)
    df.set_index("timestamp",inplace=True)
    globals_[k]=df

# Dedup token-specific
for name,df in [("holders",df_holders),("new_token_accounts",df_newacc),("transfers",df_transfers)]:
    df.sort_values(["token_mint","timestamp"],inplace=True)
    df.drop_duplicates(["token_mint","timestamp"],keep="last",inplace=True)

# Dedup OHLCV
for symbol,df in dfs_tokens.items():
    df.sort_values("timestamp",inplace=True)
    df.drop_duplicates("timestamp",keep="last",inplace=True)

# Overall span
times=[] 
for df in list(globals_.values())+[df_holders,df_newacc,df_transfers]+list(dfs_tokens.values()):
    if "timestamp" in df.columns:
        times.append(df["timestamp"].min()); times.append(df["timestamp"].max())
    else:
        times.append(df.index.min()); times.append(df.index.max())
overall_start,overall_end = min(times),max(times)
idx = pd.date_range(overall_start,overall_end,freq="12h")

# Align globals
aligned_globals={}
for name,df in globals_.items():
    df2 = df.reindex(idx).ffill().reset_index().rename(columns={"index":"timestamp"})
    aligned_globals[name]=df2

# Align holders/newacc/transfers by mint
aligned_holders,aligned_newacc,aligned_transfers={}, {}, {}
for mint,grp in df_holders.groupby("token_mint"):
    df2 = grp.set_index("timestamp")[["holder_count"]].reindex(idx).reset_index().rename(columns={"index":"timestamp"})
    aligned_holders[mint]=df2
for mint,grp in df_newacc.groupby("token_mint"):
    df2 = grp.set_index("timestamp")[["new_token_accounts"]].reindex(idx).reset_index().rename(columns={"index":"timestamp"})
    aligned_newacc[mint]=df2
for mint,grp in df_transfers.groupby("token_mint"):
    df2 = grp.set_index("timestamp")[["transfer_count"]].reindex(idx).reset_index().rename(columns={"index":"timestamp"})
    aligned_transfers[mint]=df2

# Align OHLCV per symbol
aligned_ohlcv={}
for sym,df in dfs_tokens.items():
    mint=symbol_to_mint[sym]
    df2 = df.set_index("timestamp").reindex(idx).reset_index().rename(columns={"index":"timestamp"})
    df2["token_symbol"]=sym
    df2["token_mint"]=mint
    aligned_ohlcv[sym]=df2

# =====================================================
# Step 3: Merge into Master Panel (merge on token SYMBOL)
# =====================================================

# 3.1 build masters keyed by 'token'
df_ohlcv_master = pd.concat([df.assign(token=df["token_symbol"]) for df in aligned_ohlcv.values()], ignore_index=True)
df_holders_master = pd.concat([df.assign(token=mint_to_symbol[m]) for m,df in aligned_holders.items()], ignore_index=True)
df_holders_master = df_holders_master[["token","timestamp","holder_count"]]
df_newacc_master = pd.concat([df.assign(token=mint_to_symbol[m]) for m,df in aligned_newacc.items()], ignore_index=True)
df_newacc_master = df_newacc_master[["token","timestamp","new_token_accounts"]]
df_transfers_master = pd.concat([df.assign(token=mint_to_symbol[m]) for m,df in aligned_transfers.items()], ignore_index=True)
df_transfers_master = df_transfers_master[["token","timestamp","transfer_count"]]

# 3.2 merge on token+timestamp
df_tokens_combined = (
    df_ohlcv_master
    .merge(df_holders_master, on=["token","timestamp"], how="left")
    .merge(df_newacc_master, on=["token","timestamp"], how="left")
    .merge(df_transfers_master, on=["token","timestamp"], how="left")
)

# reattach token_mint
df_tokens_combined["token_mint"] = df_tokens_combined["token"].map(symbol_to_mint)

# 3.3 merge globals
df_master = df_tokens_combined.copy()
for name,dfg in aligned_globals.items():
    rename_map = {c:f"{name}_{c}" for c in dfg.columns if c!="timestamp"}
    dfg2 = dfg.rename(columns=rename_map)
    df_master = df_master.merge(dfg2, on="timestamp", how="left")

# 3.4 reorder & save
token_cols = ["timestamp","token_mint","token","open_usd","high_usd","low_usd","close_usd","volume_usd","holder_count","new_token_accounts","transfer_count"]
global_cols = [c for c in df_master.columns if c not in token_cols]
df_master = df_master[token_cols+global_cols]
df_master.to_parquet(os.path.join(base_dir,"solana_master_raw.parquet"))
print("Saved solana_master_raw.parquet")


Saved solana_master_raw.parquet


In [17]:
df_master

,timestamp,token_mint,token,open_usd,high_usd,low_usd,close_usd,volume_usd,holder_count,new_token_accounts,...,btc_eth_price_eth_low,btc_eth_price_eth_close,sol_price_open,sol_price_high,sol_price_low,sol_price_close,sol_price_volume,network_tx_tx_count,tvl_tvl_usd,tvl_tvl_change_12h
0,2024-12-05 00:00:00,c7heqqfnzdmbufqwchkl9fvdwsfsdrbnfwzddywycltz,$COLLAT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,866531.0,NaN,NaN
1,2024-12-05 12:00:00,c7heqqfnzdmbufqwchkl9fvdwsfsdrbnfwzddywycltz,$COLLAT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3806.624693,3806.624693,242.483567,242.483567,232.376240,233.289091,2611.759253,182550905.0,NaN,NaN
2,2024-12-06 00:00:00,c7heqqfnzdmbufqwchkl9fvdwsfsdrbnfwzddywycltz,$COLLAT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3791.374120,3872.775968,236.882044,244.364756,235.805695,236.044413,2873.570727,183188078.0,1.941217e+10,0.028605
3,2024-12-06 12:00:00,c7heqqfnzdmbufqwchkl9fvdwsfsdrbnfwzddywycltz,$COLLAT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3854.354317,4031.584105,234.225689,240.084565,232.639141,238.073414,2852.116897,183446143.0,1.941217e+10,0.000000
4,2024-12-07 00:00:00,c7heqqfnzdmbufqwchkl9fvdwsfsdrbnfwzddywycltz,$COLLAT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3986.505629,3986.815519,237.089680,238.549845,235.117627,238.438472,2837.946730,183675936.0,1.971885e+10,0.015798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8321,2025-06-01 12:00:00,8x5vqbha8d7nkd52unus5nnt3pwa8pld34ymskeso2wn,ZEREBRO,0.029810,0.037385,0.029327,0.035637,0.390691,73697.0,53.0,...,2484.639706,2538.239570,152.103430,156.893238,151.033222,156.893238,1855.822749,176590346.0,1.909351e+10,0.000000
8322,2025-06-02 00:00:00,8x5vqbha8d7nkd52unus5nnt3pwa8pld34ymskeso2wn,ZEREBRO,0.036545,0.036552,0.031606,0.031826,0.403126,73704.0,25.0,...,2482.897102,2483.965452,157.678225,158.278749,153.713839,153.966326,1872.837072,172739102.0,1.918176e+10,0.004622
8323,2025-06-02 12:00:00,8x5vqbha8d7nkd52unus5nnt3pwa8pld34ymskeso2wn,ZEREBRO,0.031822,0.032946,0.030969,0.032946,0.384153,73704.0,20.0,...,2483.016657,2581.057455,153.718959,156.300049,152.574077,156.300049,1845.678881,179729332.0,1.918176e+10,0.000000
8324,2025-06-03 00:00:00,8x5vqbha8d7nkd52unus5nnt3pwa8pld34ymskeso2wn,ZEREBRO,0.032649,0.034154,0.032649,0.033673,0.404068,73692.0,15.0,...,2596.048973,2614.005891,156.878012,161.951983,156.878012,161.158466,1914.444299,160288119.0,1.952288e+10,0.017783


In [23]:
# Checking that transfer_count is exists

import pandas as pd

# 1) Load your master panel
df = pd.read_parquet(r"C:\Users\james\OneDrive\Documents\GitHub\solana-qrf-interval-forecasting\data\solana_master_raw.parquet")

# 2) View the raw transfer_count column (all rows, including NaNs)
print(df["transfer_count"])

# 3) If you only want the non‐null values (i.e. where transfers actually happened):
print(df["transfer_count"].dropna())

# 4) To see transfer_count together with its token & timestamp:
print(
    df[["timestamp","token","transfer_count"]]
      .dropna(subset=["transfer_count"])
      .sort_values(["token","timestamp"])
      .reset_index(drop=True)
)

# 5) Quick summary stats on transfer_count:
print(df["transfer_count"].describe())


0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
         ...  
8321    4114.0
8322    2130.0
8323     280.0
8324     117.0
8325       NaN
Name: transfer_count, Length: 8326, dtype: float64
113      249.0
114      152.0
115      157.0
116       39.0
117       27.0
         ...  
8320     306.0
8321    4114.0
8322    2130.0
8323     280.0
8324     117.0
Name: transfer_count, Length: 7502, dtype: float64
               timestamp    token  transfer_count
0    2025-01-30 12:00:00  $COLLAT           249.0
1    2025-01-31 00:00:00  $COLLAT           152.0
2    2025-01-31 12:00:00  $COLLAT           157.0
3    2025-02-01 00:00:00  $COLLAT            39.0
4    2025-02-01 12:00:00  $COLLAT            27.0
...                  ...      ...             ...
7497 2025-06-01 00:00:00  titcoin         21771.0
7498 2025-06-01 12:00:00  titcoin         14316.0
7499 2025-06-02 00:00:00  titcoin         15871.0
7500 2025-06-02 12:00:00  titcoin          2229.0
7501 2025-06-03 00